In [6]:
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import StructuredTool
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# def plus(a, b):
#     return a + b


# # structured input ReAct(Reasoning and Acting) -> not related to reactjs lol
# # * how an agent works like a loop and langchain does pass the context.
# # langchane behaves like a bridge -> sometimes it can be broken
# # -> better option: OpenAI function calling -> force the output
# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     tools=[
#         StructuredTool.from_function(
#             func=plus,
#             name="Sum Calculator",
#             description="A calculator that adds two numerical values and returns their sum. Both arguments must be numbers.",
#         )
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# # llm.invoke(prompt)
# agent.run(prompt)
# # agent.invoke({"input": prompt})

In [7]:
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import Tool
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# def plus(inputs):
#     a, b = inputs.split(",")
#     return float(a) + float(b)


# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     # Deprecated since version 0.1.0: LangChain agents will continue to be supported,
#     # but it is recommended for new use cases to be built with LangGraph.
#     # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     # It works well now without setting True
#     handle_parsing_errors=True,
#     tools=[
#         Tool.from_function(
#             func=plus,
#             name="Sum Calculator",
#             description="A calculator that adds two numerical values and returns their sum. Use this tool by sending a pair of numbers separated by a comma. \n Example:1,2",
#         )
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# # llm.invoke(prompt)
# agent.run(prompt)
# # agent.invoke({"input": prompt})

In [8]:
# from typing import Type
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import BaseTool
# from pydantic import BaseModel, Field
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# # define args types by using pydantic
# class CalculatorToolArgsSchema(BaseModel):
#     # if set int, langchain automatically rounds values
#     a: float = Field(discription="The first number")
#     b: float = Field(discription="The second number")


# class CalculatorTool(BaseTool):

#     name = "CalculatorTool"
#     description = """
#     Use this to perform sums of two numbers.
#     the first and second arguments must be numbers
#     only receive two numbers.
#     """
#     args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

#     # now the data structure is simple, but if prepare for bit data structures,
#     # separating function is better than just repeat a + b in return
#     def _run(self, a, b):
#         return a + b


# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     # Deprecated since version 0.1.0: LangChain agents will continue to be supported,
#     # but it is recommended for new use cases to be built with LangGraph.
#     # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html
#     agent=AgentType.OPENAI_FUNCTIONS,
#     # It works well now without setting True
#     handle_parsing_errors=True,
#     tools=[
#         CalculatorTool(),
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# agent.invoke(prompt)

In [9]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol of a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
"""
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        wrapper = DuckDuckGoSearchAPIWrapper(
            region="wt-wt",  # Default global region
            time="y",  # Search period (y: 1 year)
            max_results=5,  # Maximum number of results
        )
        search = DuckDuckGoSearchResults(
            api_wrapper=wrapper,
            backend="html",
        )
        return search.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information of Cloudflare's stock and help me help me analyze if it's a potential good investment. And also tell me what stock symbol does the stock have."


agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


[snippet: Stay on top of changing Australian Stock Markets with our exclusive expert Analyst Reports. Stocks buy, sell & hold recommendations with detailed fundamental and technical analysis., title: Best Small Caps Stocks ASX - Download Our Free Report Today, link: https://duckduckgo.com/y.js?ad_domain=sharesinvalue.com.au&ad_provider=bingv7aa&ad_type=txad&click_metadata=fRmJB9Ws7OlMrsmDFjbewOSd2AAD7QW-I44mLaX4CavUwqRvtG4Jt9rjEPix1jfgQGbpxvxNPzDeMBiQ-da87YRdJmZuUpn28H0Drlv0L30ccLtxJGn0Jjh65wpRbZfT.Xr1m35I0I7t4_ZCatu-JFw&rut=73cabae5cae97f4f5de1f22cbcb4a9e6a429b6b09348daec9220eca70b8a2a3c&u3=https://www.bing.com/aclick?ld=e8MiPzRJ0LWWrty4T-MexJEjVUCUz6O4rvfY2rnoZJPpSUAfrFHWbS71W8clpeEQTag8asnuaqX0YEZL9n3DrjYFhRKv97MxipC7yECUUBJW7ifzM5pFR1OkNY2CSD5TzaXhhdfFfmV7En2yQ5pTYX58YnGr8x2PEo7ThMA67BhoowugeWL1IsJOKVZT7z1Awg8oVqVw&u=aHR0cHMlM2ElMmYlMmZ3d3cuc2hhcmVzaW52YWx1ZS5jb20

{'input': "Give me information of Cloudflare's stock and help me help me analyze if it's a potential good investment. And also tell me what stock symbol does the stock have.",
 'output': "Cloudflare's stock symbol is NET. Cloudflare, Inc. operates as a cloud services provider that delivers a range of services to businesses worldwide. The company provides an integrated cloud-based security solution to secure a range of combination of platforms, including public cloud, private cloud, on-premises, software-as-a-service applications, and Internet of things (IoT) devices; and website and application services, such as content delivery network (CDN), Internet security, load balancing, video and image optimization, Web application firewall, and domain name system services.\n\nTo analyze if Cloudflare's stock is a potential good investment, you may consider the following factors:\n1. Financial Performance: Look at the company's revenue growth, profitability, and cash flow.\n2. Market Position: 